In [1]:
from multiprocessing.sharedctypes import Value
import networkx as nx
import numpy as np
import random
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import pickle

# File imports
import data
from data import select_dataset, get_station_g, ingest_electricity_data, set_random_speed_columns, ingest_pems
from replicate_graph import layer_graph
from vehicle import Vehicle
from simulation import Simulation
from visualization import set_draw_attributes
from vehicle_list_data import get_utilization

## Ingest pems for the whole graph (all charging and parking stops)
### commented out because it only needs to be done once. Csv already saved.

In [2]:
# all_graph = ingest_pems("data/charging_parking_stations_lonlat.csv", "data/charging_parking_distances.csv")

In [3]:
# #save the new csv with all the speeds (from pems ingest above), which will be used for the algorithm
# all_graph[1].to_csv("data/charging_parking_distances_pemsingested.csv")

## Run the algorithm

In [ ]:
number_of_iterations = 3
kwh_per_km = 1.9

#scenario list: each list inside has three params : charging rate, kwh per km, and battery capacity.
# scenario_list = [[45,1.9,215],[90,1.9,215],[45,0.95,215],[45,1.9,430],[90,0.95,430]] 
# scenario_list = [[90,0.95,430], [45,1.9,430], [45,0.95,215], [90,1.9,215], [45,1.9,215]] 
scenario_list = [[45,1.9,215]] #single iteration

columns = ["scenario_number", "iteration", "charging_rate", "km_per_kwh", "battery_capacity", "success",
           "station_utilization_disp_of_avg", "station_utilization_avg_of_disp", "electricity",
           "percent_delay", "hours_spent_in_queues", "hours_spent_charging", 
           "most_utilized_node", "least_utilized_node",
          "strongly_connected_components", "weakly_connected_components"]
results = [columns]

basename = "wednesday_debug_2"
saving_path = "trials/"+basename+"/"

def print_results(sim, scenario, s, n, success=True):
    print(success, ": name: ", sim.name)
    
    if success==True:
        print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
        print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
        print("electricity:  ", sim.metrics['electricity'])
        print("percent_delay:  ", sim.metrics['percent_delay'])
        print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
        print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])
        

for scenario, s in zip(scenario_list, range(0,len(scenario_list))):
    
    print('------- SCENARIO '+str(s)+' -------')
    #initialize for wcctici
    simulation_length = 12
    battery_interval = 20
    kwh_per_km = scenario[1]
    battery_capacity = scenario[2]
    stations_path = "data/wcctci_stations-updated.csv"
    distances_path = "data/wcctci_coord_distances.csv"
    
    for n in range(0, number_of_iterations):
        
        #reset the variables for the next iteration
        if n==0:
            stations_path = "data/wcctci_stations-updated.csv"
            distances_path = "data/wcctci_coord_distances.csv"
        else:
            stations_path = "data/algorithm_"+str(n)+"_stations.csv"
            distances_path = "data/algorithm_"+str(n)+"_distances.csv"
        name = basename+"_scenario"+str(s)+"_algorithm_"+str(n)
        
        ### run and print metrics
        sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity, saving_path)
        sim.add_demand_nodes()
        success = True
        try:
            sim.run()
            print_results(sim, scenario, s, n, success)
        except Exception as e: 
            print(e)
            success = False
            print_results(sim, scenario, s, n, success)
            continue # don't attempt data analysis on failed iteration, but continue loop

        ### get station utilization

        #get the station ids in order 
        current_stations = [int(j) for j in sim.station_g.nodes]

#         #get the utilization rate of the stations
#         utilization = sim.metrics['station_utilization']
        
        # Alternative utilization calculation 
        utilization = get_utilization(sim)
        
        avg_utilization = {}
        for node in current_stations:
            avg_utilization[node] = 0
            for i in range(len(utilization)):
                use_i = utilization[i]
                try:
                    avg_utilization[node] += use_i[str(node)]
                except:
                    avg_utilization[node] += 0

            avg_utilization[node] = avg_utilization[node]/len(utilization)

        utilization = [i for i in avg_utilization.values()]

        #get the station that has the highest utilization.
        h_st = current_stations[np.argmax(utilization)]

        #get the station that has the lowest utilization.
        l_st = current_stations[np.argmin(utilization)]

        print("Highest:", h_st)
        print("Lowest:", l_st)
        print("________________________________________")
        
        # record the results (including highest + lowest utilized nodes)
        if success == True:
            result = [s, n, scenario[0], scenario[1], scenario[2], True,
            sim.metrics['station_utilization_disp_of_avg'], sim.metrics['station_utilization_avg_of_disp'], 
            sim.metrics['electricity'], sim.metrics['percent_delay'],
            sim.metrics['hours_spent_in_queues'], sim.metrics['hours_spent_charging'],
            h_st, l_st, 
           nx.number_strongly_connected_components(sim.battery_g), nx.number_weakly_connected_components(sim.battery_g)]
        else:
            result = [s, n, scenario[0], scenario[1], scenario[2], False, None, None, None, None, None, None, None, None, None, None]
        results.append(result)
        
        # If that was the last iteration, do not do prep for next iteration
        if n==number_of_iterations-1:
            break
        
        ### Find the unused location that is closest to the highly utilized station

        #get the map of all parking areas and stations
        full_map_dist = pd.read_csv("data/charging_parking_distances_pemsingested.csv")
        full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

        #keep only the roads that start or end with the high utilization station
        h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

        #then we eliminate all of the stations that are already being used.
        using_st = current_stations.copy()
        using_st.remove(h_st)
        h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

        #then we remove the case where we go from h_st to h_st
        h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

        #find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

        new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
        new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

        #one of the two will be h_st and the other will be the new station location. 
        if new_origin == h_st:
            new_station = new_destination
        else:
            new_station = new_origin

        print("New Station:", new_station)
        ### create the new maps with the locations and the distances of the stations that we will be using next

        new_stations_list = current_stations + [new_station]
        new_stations_list.remove(l_st)

        #now we reduce these to the new set of stations. All origins and destinations should be in this list.
        new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
        new_map_loc = full_map_loc[full_map_loc.loc[:, 'OID_'].isin(new_stations_list)]

        #we add the charging rates and capacity- assuming that they are the same as the others (set to 45 and 8)
        #TO DO: check this
        new_map_loc['charging_rate'] = scenario[0]
        new_map_loc['physical_capacity'] = 8

        #we change the snapX and snapY columns to lon and lat 
        new_map_loc = new_map_loc.rename(columns = {'SnapX': 'longitude', 'SnapY': 'latitude'})

        #now we save them. 
        new_map_loc.to_csv("data/algorithm_"+str(n+1)+"_stations.csv") 
        new_map_dist.to_csv("data/algorithm_"+str(n+1)+"_distances.csv")
        

pd.DataFrame(results).to_csv(saving_path+basename+".csv")

------- SCENARIO 0 -------


100%|██████████| 12/12 [01:33<00:00,  7.82s/it]


True : name:  wednesday_debug_2_scenario0_algorithm_0
station_utilization_disp_of_avg:   43.90078125
station_utilization_avg_of_disp:   44.14869791666667
electricity:   0
percent_delay:   (1.0, 1.0, 4.529187227438623, 1.2179298304197967, 0.6901577502575181)
hours_spent_in_queues:   (0.0, 0.0, 6.2, 0.30405405405405406, 1.0831559480928725)
hours_spent_charging:   (0, 0.0, 11, 1.0337837837837838, 2.980602924089082)
Highest: 37
Lowest: 1
________________________________________
New Station: 292


/Users/alandler/.local/lib/python3.7/site-packages/ipykernel_launcher.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/alandler/.local/lib/python3.7/site-packages/ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 12/12 [01:14<00:00,  6.19s/it]


True : name:  wednesday_debug_2_scenario0_algorithm_1
station_utilization_disp_of_avg:   37.76510416666667
station_utilization_avg_of_disp:   37.8390625
electricity:   0
percent_delay:   (1.0, 1.0, 5.453842132056268, 1.2855760066646609, 0.8155778809344603)
hours_spent_in_queues:   (0.0, 0.0, 6.4, 0.31142857142857144, 1.0704528890871856)
hours_spent_charging:   (0, 0.0, 11, 1.0357142857142858, 2.9985966105251736)
Highest: 16
Lowest: 2
________________________________________
New Station: 210


 75%|███████▌  | 9/12 [00:47<00:18,  6.32s/it]

In [ ]:
utilization

In [ ]:
# avg_utilization = {}
# for node in current_stations:
#     avg_utilization[node] = 0
#     for i in range(len(utilization)):
#         use_i = utilization[i]
#         try:
#             avg_utilization[node] += use_i[str(node)]
#         except:
#             avg_utilization[node] += 0
    
#     avg_utilization[node] = avg_utilization[node]/len(utilization)

# utilization_list = [i for i in avg_utilization.values()]

## Bellow this is  just code I used to think about/ build the algorithm above (Just ignore).
### I was thinking about just one iteration, starting from the wcctci pickle.

In [ ]:
#open the pickle
with open('trials/wcctci_04_04_2022_12_15_28.pkl', 'rb') as inp:
    res = pickle.load(inp)

Get the highest and lowest utilized stations:

In [ ]:
#get station utilization

#get the station ids in order 
current_stations = list(res.station_g.nodes)

#get the utilization rate of the stations
utilization = res.metrics['station_utilization']

#get the station that has the highest utilization.
h_st = current_stations[np.argmax(utilization)]

#get the station that has the lowest utilization.
l_st = current_stations[np.argmin(utilization)]

Find the unused location that is closest to the highly utilized station:

In [ ]:
#get the map of all parking areas and stations
full_map_dist = pd.read_csv("data/charging_parking_distances.csv")

#keep only the roads that start or end with the high utilization station
h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

#then we eliminate all of the stations that are already being used.
using_st = current_stations.copy()
using_st.remove(h_st)
h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

#then we remove the case where we go from h_st to h_st
h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

#find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

#one of the two will be h_st and the other will be the new station location. 
if new_origin == h_st:
    new_station = new_destination
else:
    new_station = new_origin

Now we start to think about re-runing the simulation with this swap (taking out l_st and adding new_station). To do this, we need to create a new map with the locations and the distances of the stations that we will be using next.

In [ ]:
new_stations_list = current_stations + [new_station]

In [ ]:
#we have "full_map_dist" (distances map)
#and we need the locations map
full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

#now we reduce these to the new set of stations. All origins and destinations should be in this list.
new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
new_map_loc = full_map_loc[(full_map_loc.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_loc.loc[:, 'DestinationID'].isin(new_stations_list))]

In [ ]:
#now we save them. 
new_map_loc.to_csv("data/algorithm_"+str(1)+"_stations.csv") ########change the 1
new_map_dist.to_csv("data/algorithm_"+str(1)+"_distances.csv")

Now we can run the next simulation

In [ ]:
simulation_length = 12
battery_interval = 20
km_per_percent = 3.13

sim = Simulation("data/algorithm_"+str(1)+"_stations.csv", "data/algorithm_"+str(1)+"_distances.csv", simulation_length, battery_interval, km_per_percent)

# Anna

In [ ]:
stations_path = "data/wcctci_stations-updated.csv"
distances_path = "data/wcctci_coord_distances.csv"
# stations_df, distances_df = select_dataset(stations_csv_path, distances_csv_path)
# station_g = get_station_g(stations_df, distances_df)
# battery_g = layer_graph(station_g, 20, km_per_percent= 1.9)
simulation_length = 24
battery_interval = 20
kwh_per_km = 1.9
battery_capacity = 215
name = 'hi'
sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
sim.add_demand_nodes()

In [ ]:
sim.battery_g.nodes

In [ ]:
nx.shortest_path(sim.battery_g, "Long Beach", "Reno")